# Angular Velocity Visual Demos (3 Examples)
This notebook is designed to **show** what angular velocity means in three settings:
1) A basic rotating frame and a vector attached to it
2) An airplane-like body showing yaw/pitch/roll and why the order matters
3) A simple “industrial gripper finger” (a small kinematic chain) showing point velocities from joint rotation

Along the way we connect to the key equations you asked about:
- **Cross product matrix**:  $x\times y = [x]y$
- **Rotation matrix kinematics**:  $\dot R_{sb} = [\omega_s]R_{sb}$
- **Same angular velocity in different coordinates**:  $\omega_b = R_{sb}^T\,\omega_s$

> You can tweak parameters (angles, angular velocities) in the code cells and rerun to build intuition.

## Dimensions and Exponential / Logarithm Intuition:  so(3) vs SO(3)

### Dimensions: so(3) vs SO(3)

**so(3)** (little so) is the set of all 3×3 **skew-symmetric matrices**.

A general element looks like:
\[
\begin{bmatrix}
0 & -a_3 & a_2 \\
a_3 & 0 & -a_1 \\
-a_2 & a_1 & 0
\end{bmatrix}
\]

It is completely determined by the three numbers \((a_1, a_2, a_3)\).

- ✅ **Dimension = 3**
- It is a **3D linear vector space**
- Elements represent **infinitesimal rotations / angular velocity generators**

---

**SO(3)** (big SO) is the set of all 3×3 **rotation matrices**:
\[
R^T R = I, \quad \det(R)=1
\]

Although a rotation matrix has 9 entries, the orthonormality constraints reduce the degrees of freedom to 3.

- ✅ **Dimension = 3**
- It is a **3D curved space (manifold / Lie group)**
- Elements represent **finite rotations**

---

### Same dimension, different meaning

Both spaces are 3-dimensional, but in **different senses**:

- **so(3)**: 3D **linear** space (you can add and scale elements freely)
- **SO(3)**: 3D **nonlinear / curved** space (you cannot add rotations directly)

You should think of:
- so(3) as a **tangent space**
- SO(3) as the **rotation manifold**

---

## Scalar Exponential Analogy

For a scalar system:
\[
\dot x(t) = a x(t)
\quad \Rightarrow \quad
x(t) = e^{a t} x(0)
\]

Here:
- \(a\) is an **infinitesimal generator**
- \(e^{at}\) accumulates that generator into a **finite effect**

The exponential map connects:
\[
a \;\longleftrightarrow\; e^{a}
\]

---

## Rotation Exponential Analogy

For rotations, the exact same structure appears:

\[
\dot R(t) = [\omega] R(t)
\quad \Rightarrow \quad
R(t) = e^{[\omega] t} R(0)
\]

Here:
- \([\omega] \in \mathfrak{so}(3)\) is an **infinitesimal rotation**
- \(e^{[\omega] t} \in SO(3)\) is a **finite rotation**

So the direct analogy is:

- **scalar**: \(a \;\leftrightarrow\; e^{a}\)
- **rotation**: \([\omega]\theta \;\leftrightarrow\; e^{[\omega]\theta}\)

This is why rotations are described using **matrix exponentials**.

---

## Exponential Coordinates of Rotation

A rotation can be compactly represented by:
\[
[\hat\omega]\theta \in \mathfrak{so}(3)
\]

where:
- \(\hat\omega\) is a unit rotation axis
- \(\theta\) is the rotation angle

The corresponding rotation matrix is:
\[
R = \exp([\hat\omega]\theta)
\]

This representation is called **exponential coordinates of rotation**.

---

## Logarithm Analogy (Inverse Map)

Just as the scalar logarithm inverts the exponential:

\[
x \;\longleftrightarrow\; \log(x)
\]

we have a **matrix logarithm** for rotations:

\[
\log : SO(3) \rightarrow \mathfrak{so}(3)
\]

That is:
- **exp** maps an infinitesimal generator to a rotation
- **log** extracts the infinitesimal generator from a rotation

\[
\exp: [\hat\omega]\theta \in \mathfrak{so}(3) \;\rightarrow\; R \in SO(3)
\]
\[
\log: R \in SO(3) \;\rightarrow\; [\hat\omega]\theta \in \mathfrak{so}(3)
\]

---

## Differentiation Viewpoint (Why Angular Velocity Lives in so(3))

Angular velocity lives in **so(3)** because it is the **time derivative of orientation**:

\[
[\omega] = \dot R R^T
\]

- \(\dot R\) tells you how orientation is changing
- Multiplying by \(R^T\) brings that change back to the tangent space
- The result is always **skew-symmetric**, hence in so(3)

So:
- **SO(3)** = configurations (where you are)
- **so(3)** = velocities / infinitesimal motions (how you move)

---

### One-line mental model

> **so(3)** contains infinitesimal spins  
> **SO(3)** contains finite orientations  
> **exp** integrates spins into rotations  
> **log** differentiates rotations back into spins


## so(3) vs SO(3) + exp/log intuition (short)

### Dimensions
- **so(3)**: all 3×3 **skew-symmetric** matrices. Determined by 3 numbers ⇒ **dim = 3** (a 3D *vector space*).
- **SO(3)**: all 3×3 **rotation matrices** \(R^TR=I,\ \det R=1\). 3 degrees of freedom ⇒ **dim = 3** (a 3D *manifold / Lie group*).

### Scalar vs rotation analogy
- **Scalar**: \(\dot x = a x \Rightarrow x(t)=e^{at}x(0)\) and \(a \leftrightarrow e^a\).
- **Rotation**: \(\dot R = [\omega]R \Rightarrow R(t)=e^{[\omega]t}R(0)\) and \([\omega]\theta \leftrightarrow e^{[\omega]\theta}\).

### exp / log maps
- \(\exp: \ \mathfrak{so}(3)\to SO(3)\), \(\ R=\exp([\hat\omega]\theta)\) (axis-angle → rotation matrix).
- \(\log: \ SO(3)\to \mathfrak{so}(3)\), \(\ \log(R)=[\hat\omega]\theta\) (rotation matrix → axis-angle generator).

### Differentiation viewpoint (why ω lives in so(3))
\[
[\omega] = \dot R R^T \in \mathfrak{so}(3)
\]
so **SO(3)** is “where you are” (orientation) and **so(3)** is “how you move” (instantaneous rotation / angular velocity).


In [ ]:
# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

np.set_printoptions(precision=4, suppress=True)

## Helpers

In [ ]:
def norm(v): 
    return float(np.linalg.norm(v))

def unit(v, eps=1e-12):
    v = np.asarray(v, dtype=float).reshape(-1)
    n = np.linalg.norm(v)
    if n < eps: return np.zeros_like(v)
    return v/n

def skew(w):
    w1,w2,w3 = np.asarray(w).reshape(3,)
    return np.array([[0, -w3, w2],
                     [w3, 0, -w1],
                     [-w2, w1, 0]], dtype=float)

def setup_3d(ax, lim=1.5, title=None):
    ax.set_xlim([-lim, lim]); ax.set_ylim([-lim, lim]); ax.set_zlim([-lim, lim])
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    ax.set_box_aspect([1,1,1])
    if title: ax.set_title(title)

def draw_vec(ax, v, label=None, origin=(0,0,0), linewidth=2):
    v = np.asarray(v).reshape(3,)
    ox, oy, oz = origin
    ax.quiver(ox, oy, oz, v[0], v[1], v[2], arrow_length_ratio=0.08, linewidth=linewidth)
    if label:
        ax.text(ox+v[0], oy+v[1], oz+v[2], " "+label, fontsize=10)

def Rx(a):
    c,s = np.cos(a), np.sin(a)
    return np.array([[1,0,0],[0,c,-s],[0,s,c]], float)

def Ry(a):
    c,s = np.cos(a), np.sin(a)
    return np.array([[c,0,s],[0,1,0],[-s,0,c]], float)

def Rz(a):
    c,s = np.cos(a), np.sin(a)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]], float)

def q_from_axis_angle(axis, angle):
    axis = unit(axis)
    half = angle/2.0
    return np.array([np.cos(half), *(np.sin(half)*axis)], float)

def q_mul(q1, q2):
    w1,x1,y1,z1 = q1
    w2,x2,y2,z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ], float)

def q_conj(q):
    w,x,y,z = q
    return np.array([w,-x,-y,-z], float)

def q_unit(q):
    return q / (np.linalg.norm(q)+1e-12)

def q_rotate(q, v):
    q = q_unit(q)
    vq = np.array([0.0, *v], float)
    return q_mul(q_mul(q, vq), q_conj(q))[1:]

def R_from_q(q):
    q = q_unit(q)
    w,x,y,z = q
    return np.array([
        [1-2*(y*y+z*z),   2*(x*y - w*z),   2*(x*z + w*y)],
        [2*(x*y + w*z),   1-2*(x*x+z*z),   2*(y*z - w*x)],
        [2*(x*z - w*y),   2*(y*z + w*x),   1-2*(x*x+y*y)]
    ], float)

## Key equations (the ones you asked about)

### 1) Cross product matrix
For any $x,y\in\mathbb{R}^3$:
$$x\times y = [x]y$$
where $[x]$ is the **skew-symmetric matrix** built from $x$.

### 2) Rotation matrix kinematics
Let $R_{sb}(t)$ map body coordinates to space coordinates. Its columns are the body axes written in space.
If the body has angular velocity $\omega_s$ (expressed in space coordinates), then:
$$\dot R_{sb} = [\omega_s] R_{sb}$$

### 3) Same physical $\omega$ in different coordinates
$\omega$ is a geometric vector, but its **components** depend on the frame:
$$\omega_b = R_{sb}^T\,\omega_s$$
(because $R_{sb}^{-1}=R_{sb}^T$ for rotations).

### What $\omega$ means (definition you can use)
For any vector $v$ rigidly attached to the body, expressed in the space frame:
$$\dot v = \omega_s \times v$$
That’s the clean *operational definition* of angular velocity.


## Demo 1 — Basic 3D: rotating frame + tangent direction
We rotate a body about a fixed axis with constant $\omega_s$ and show:
- the path of a body-fixed vector $v_b$ as seen in space: $v_s(t)=R_{sb}(t)v_b$
- the tangent direction $\dot v = \omega_s\times v$


In [ ]:
# --- Parameters to tweak ---
omega_s = np.array([0.2, 0.5, 1.2])   # angular velocity expressed in space (rad/s)
v_b = unit([1.0, 0.2, 0.0])           # a vector fixed in the body frame

dt = 0.02
T = 2.0
ts = np.arange(0, T+dt, dt)

R = np.eye(3)
traj_v = []
traj_vdot = []
for _t in ts:
    v_s = R @ v_b
    vdot = np.cross(omega_s, v_s)
    traj_v.append(v_s)
    traj_vdot.append(vdot)

    # small-step update: R <- exp([omega] dt) R
    w = omega_s
    ang = norm(w)*dt
    if ang < 1e-12:
        dR = np.eye(3)
    else:
        axis = w/(norm(w)+1e-12)
        K = skew(axis)
        dR = np.eye(3) + np.sin(ang)*K + (1-np.cos(ang))*(K@K)
    R = dR @ R

traj_v = np.array(traj_v)
traj_vdot = np.array(traj_vdot)

print("Check identity: vdot ≈ [omega] v")
print("max error:", np.max(np.linalg.norm(traj_vdot - (skew(omega_s)@traj_v.T).T, axis=1)))

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="3d")
setup_3d(ax, lim=1.3, title="Demo 1: v(t) path and tangent vdot = ω×v")

draw_vec(ax, omega_s, "ω_s")
ax.plot(traj_v[:,0], traj_v[:,1], traj_v[:,2])

for k in [0, len(ts)//2, -1]:
    draw_vec(ax, traj_v[k], f"v(t={ts[k]:.1f})", linewidth=2)
    draw_vec(ax, 0.25*traj_vdot[k]/(norm(traj_vdot[k])+1e-12), "tangent (scaled)", origin=traj_v[k], linewidth=1)

plt.show()

## Demo 2 — Airplane-style: yaw/pitch/roll + estimating ω from motion
We show two different Euler composition orders (they differ), then apply a small extra rotation and recover an estimate of $\omega_s$ using:
$$\omega_s^\wedge \approx \dot R\,R^T$$
where $\omega^\wedge$ is the skew matrix.


In [ ]:
yaw = np.deg2rad(35)
pitch = np.deg2rad(20)
roll = np.deg2rad(-15)

R1 = Rz(yaw) @ Ry(pitch) @ Rx(roll)
R2 = Rx(roll) @ Ry(pitch) @ Rz(yaw)

x1,y1,z1 = R1[:,0], R1[:,1], R1[:,2]
x2,y2,z2 = R2[:,0], R2[:,1], R2[:,2]

fig = plt.figure(figsize=(10,5))
axA = fig.add_subplot(121, projection="3d")
axB = fig.add_subplot(122, projection="3d")
setup_3d(axA, lim=1.2, title="R = Rz(yaw) Ry(pitch) Rx(roll)")
setup_3d(axB, lim=1.2, title="R = Rx(roll) Ry(pitch) Rz(yaw)")

for ax in [axA, axB]:
    draw_vec(ax, [1,0,0], "x_s", linewidth=1)
    draw_vec(ax, [0,1,0], "y_s", linewidth=1)
    draw_vec(ax, [0,0,1], "z_s", linewidth=1)

draw_vec(axA, x1, "x_b (nose)")
draw_vec(axA, y1, "y_b (wing)")
draw_vec(axA, z1, "z_b (up)")
axA.plot([0, x1[0]], [0, x1[1]], [0, x1[2]])

draw_vec(axB, x2, "x_b (nose)")
draw_vec(axB, y2, "y_b (wing)")
draw_vec(axB, z2, "z_b (up)")
axB.plot([0, x2[0]], [0, x2[1]], [0, x2[2]])

plt.show()

print("Angle between 'nose' directions for the two orders (deg):",
      np.degrees(np.arccos(np.clip(np.dot(x1,x2)/(norm(x1)*norm(x2)), -1, 1))))

# Estimate ω_s from a small incremental rotation in SPACE
dt = 0.02
omega_s_true = np.array([0.0, 0.8, 0.2])
ang = norm(omega_s_true)*dt
axis = omega_s_true/(norm(omega_s_true)+1e-12)
K = skew(axis)
dR = np.eye(3) + np.sin(ang)*K + (1-np.cos(ang))*(K@K)

R_next = dR @ R1
Rdot_est = (R_next - R1)/dt
omega_hat_est = Rdot_est @ R1.T   # ≈ [ω_s]
omega_s_est = np.array([omega_hat_est[2,1], omega_hat_est[0,2], omega_hat_est[1,0]])

print("\nTrue ω_s:", omega_s_true)
print("Est  ω_s:", omega_s_est)
print("Skew-sym check ||ω^ + (ω^)^T||:", norm(omega_hat_est + omega_hat_est.T))

## Demo 3 — Simple industrial gripper finger: point velocity from joint ω
A point’s instantaneous velocity from a revolute joint is:
$$v = \omega \times (p - p_{joint})$$
We model a 2-link finger and compute fingertip velocity as the sum of contributions from both joints.


In [ ]:
L1, L2 = 0.7, 0.5
q1 = np.deg2rad(35)
q2 = np.deg2rad(50)

q1dot = 1.2
q2dot = -0.8

w1 = np.array([0,0,q1dot])   # base joint ω
w2 = np.array([0,0,q2dot])   # middle joint ω

p0 = np.array([0.0, 0.0, 0.0])
p1 = p0 + np.array([L1*np.cos(q1), L1*np.sin(q1), 0.0])
p2 = p1 + np.array([L2*np.cos(q1+q2), L2*np.sin(q1+q2), 0.0])

v_tip_1 = np.cross(w1, (p2 - p0))
v_tip_2 = np.cross(w2, (p2 - p1))
v_tip = v_tip_1 + v_tip_2

print("p_tip =", p2)
print("v_tip from joint1 =", v_tip_1)
print("v_tip from joint2 =", v_tip_2)
print("total v_tip =", v_tip)

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection="3d")
setup_3d(ax, lim=1.2, title="Demo 3: 2-link finger and fingertip velocity v = ω×r")

ax.plot([p0[0], p1[0]], [p0[1], p1[1]], [0,0], linewidth=3)
ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [0,0], linewidth=3)
ax.scatter([p0[0], p1[0], p2[0]], [p0[1], p1[1], p2[1]], [0,0,0], s=30)

draw_vec(ax, unit([0,0,1]), "joint axis (+z)", linewidth=1)
scale = 0.25/(norm(v_tip)+1e-12)
draw_vec(ax, scale*v_tip, "v_tip (scaled)", origin=p2, linewidth=2)

plt.show()

## Mini recap
- $\omega$ describes **how the body rotates** (axis + rate).
- Tangent/derivative of an attached vector: $\dot v = \omega\times v$.
- Orientation dynamics: $\dot R_{sb} = [\omega_s]R_{sb}$.
- Point velocity from rotation about a joint/origin: $v = \omega\times r$ (plus translation if the origin moves).
